# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part I: Preprocessing

In this tutorial, we will walk through the process of using `Snorkel` to identify mentions of spouses in a corpus of news articles. The tutorial is broken up into 5 notebooks, each covering a step in the pipeline:
1. Preprocessing
2. Candidate Extraction
3. Annotating Evaluation Data
4. Featurization & Training
5. Evaluation

In this notebook, we preprocess several documents using `Snorkel` utilities, parsing them into a simple hierarchy of component parts of our input data, which we refer to as _contexts_. We also extract standard linguistic features from each context which will be useful downstream using [CoreNLP](http://stanfordnlp.github.io/CoreNLP/), 

All of this preprocessed input data is saved to a database.  (Connection strings can be specified by setting the `SNORKELDB` environment variable.  In Snorkel, if no database is specified, then a SQLite database at `./snorkel.db` is created by default--so no setup is needed here!

### Initializing a `SnorkelSession`

First, we initialize a `SnorkelSession`, which manages a connection to a database automatically for us, and will enable us to save intermediate results.  If we don't specify any particular database (see commented-out code below), then it will automatically create a SQLite database in the background for us:

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

# Here, we just set a global variable related to automatic testing- you can safely ignore this!
max_docs = 50 if 'CI' in os.environ else float('inf')

## Loading the Corpus

Next, we load and pre-process the corpus of documents.

### Configuring a `DocPreprocessor`

We'll start by defining a `TSVDocPreprocessor` class to read in the documents, which are stored in a tab-seperated value format as pairs of document names and text.

In [2]:
from snorkel.parser import TSVDocPreprocessor

doc_preprocessor = TSVDocPreprocessor('data/articles2.tsv', max_docs=max_docs)

### Running a `CorpusParser`

We'll use an NLP preprocessing tool to split our documents into sentences, tokens, and provide annotations---part-of-speech tags, dependency parse structure, lemmatized word forms, named entities, etc.---for these sentences.

Let's run it single-threaded first; **note that this may take around 10 minutes depending on your machine**:

In [4]:
from snorkel.parser import CorpusParser
from snorkel.parser import Spacy,StanfordCoreNLPServer

#parser = Spacy(lang="en")
parser = StanfordCoreNLPServer(version='3.6.0', num_threads=4)

corpus_parser = CorpusParser(parser)
%time corpus_parser.apply(doc_preprocessor)

NameError: name 'StanfordCoreNLPServer' is not defined

We can then use simple database queries (written in the syntax of [SQLAlchemy](http://www.sqlalchemy.org/), which Snorkel uses) to check how many documents and sentences were parsed:

In [ ]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

### Running in parallel

Note that any time we execute a `UDFRunner` like `CorpusParser`, we can also execute it in parallel by running e.g.:
```python
corpus_parser.apply(doc_preprocessor, parallelism=20)
```
**Note, however, that this parallel execution will not work with SQLite, the database system used by default**; you will need to use e.g. Postgres for this!

Next, in Part 2, we will look at how to extract `Candidate` relations from our saved corpus.